In [ ]:
from __future__ import print_function
import os, shutil
import argparse

import cv2
import rosbag
from rosbag import Bag
from sensor_msgs.msg import Image
from cv_bridge import CvBridge
from tqdm import tqdm
from glob import glob
import roslz4

import pandas as pd
import numpy as np

In [ ]:
#bag_dir = '/media/minjee/4970a4b3-9bec-42aa-8022-ddff6e7b8f80/savedfiles/'
#img_dir = '/media/minjee/4970a4b3-9bec-42aa-8022-ddff6e7b8f80/images/'
bag_dir = '/data/handhygiene/bagfiles/'
img_dir = '/data/handhygiene/images'
excel_path = '/data/handhygiene/hh_video_data.xlsx'
df_ex = pd.read_excel(excel_path)
bags = sorted(glob(bag_dir + '*.bag'), key=os.path.getmtime)
add_data_from = 0

In [ ]:
def set_camera_info(i, rosbag):
    # get fps, height and width from camera info topic
    fps = 0
    height = 0
    width = 0
    info='/device_0/sensor_1/Color_0/info'
    camera_info='/device_0/sensor_1/Color_0/info/camera_info'
    
    for _, msg, _ in rosbag.read_messages(topics=[info]):
        fps = msg.fps
        if fps != 0:
            break
        
    for _, msg, _ in rosbag.read_messages(topics=[camera_info]):
        height = msg.height
        width = msg.width
        if height != 0 and width != 0:
            break
    
    print(i, fps, height, width)
    df_ex.at[i, 'fps'] = fps
    df_ex.at[i, 'height'] = height
    df_ex.at[i, 'width'] = width
    
    return 

In [ ]:
for i, bag in enumerate(tqdm(bags)):
    rosbag = Bag(bag, "r", allow_unindexed=True)
    bridge = CvBridge()
    
    if add_data_from == 0:
        data_num = i+1
    else :
        data_num = add_data_from
        
    set_camera_info(data_num-1, rosbag)
    
    ### we're only dealing with rgb images for now
    
    #depth_image_data='/device_0/sensor_0/Depth_0/image/data'
    #infrared_image_data='/device_0/sensor_0/Infrared_1/image/data'
    rgb_image_data='/device_0/sensor_1/Color_0/image/data'
    
    count = 0
    for topic, msg, t in tqdm(rosbag.read_messages(topics=[rgb_image_data])):
        
        if msg is not None:
            desired_encoding = "bgr8" if msg.encoding == "rgb8" else "passthrough" 
            cv_img = bridge.imgmsg_to_cv2(msg, desired_encoding)

            if not os.path.exists(img_dir):
                os.path.mkdir(img_dir)
            
            date = bag.split('/')[-1].split('_')[0]
            target_dir = os.path.join(img_dir, "{}_{}_frames{:06d}.jpg".format(data_num, date, count))
            cv2.imwrite(target_dir, cv_img)
            print("Wrote image {} to {}".format(count, target_dir))
            
        #shutil.move(bag, os.path.join(bag_dir, 'extracted'))
        count += 1
        
    rosbag.close()

In [ ]:
# rewrite excel
df_ex.to_excel(excel_path)

In [ ]:
#for dir in bag_sub_dirs:
#    i = 1
#    dir_rgb = dir + 'rgb_images{}'.format(i)
#    !cd $dir_rgb && pwd && mkdir $dir/videos
#    !ffmpeg -r 15 -i $dir_rgb/frame%06d.jpg -c:v libx264 -profile:v high -crf 20 -pix_fmt yuv420p $dir/videos/output.mp4 -y